**Activation Maximization on C3D**

Let's use C3D model pretrained on Sports1M dataset

In [ ]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, ZeroPadding3D
from keras.layers.core import Dense, Dropout, Flatten

model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', name='conv1', strides=(1, 1, 1), input_shape=(16, 112, 112, 3)))
model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1'))
# 2nd layer group
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same', name='conv2', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool2'))
# 3rd layer group
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3a', strides=(1, 1, 1)))
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3'))
# 4th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool4'))
# 5th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5b', strides=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5'))
model.add(Flatten())
# FC layers group
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(487, activation='softmax', name='fc8'))
model.summary()

Lets load wights and compile the model

In [ ]:
#find solution how to load weights from dropdox or google grive
model.load_weights('sports1M_weights_tf.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')

Getting indices of convolutional layers

In [2]:
layer_names = ['conv1', 'conv2', 'conv3a', 'conv3b', 'conv4a', 'conv4b', 'conv5a', 'conv5b']
layer_indices = []
for name in layer_names:
  layer_indices.append([idx for idx, layer in enumerate(model.layers) if layer.name == name][0])
layer_indices

[0, 2, 4, 5, 7, 8, 10, 11]

Getting number of filters for each convolution layer and total number of filters

In [ ]:
from vis.visualization import get_num_filters

number_of_filters = []
for idx in layer_indices:
  number_of_filters.append(get_num_filters(model.layers[idx]))
number_of_filters

total_number_of_filters = sum(number_of_filters)

Getting all activations for each layer

In [ ]:
from vis.visualization import visualize_activation

n = len(layer_indices)
#all_filters[i][j], i -- layer index incurrent array, j -- filter index
count=0
all_filters = []
for l in range(n):
  filters = []
  for f in range(number_of_filters[l]):
    filters.append(visualize_activation(model, layer_idx=layer_indices[l], filter_indices=f))
    count += 1
    print("progress: " + str(count*100.0/total_number_of_filters) + " %")
  all_filters.append(filters)

Creating gifs of activations

In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import ImageSequenceClip

n = len(layer_indices)
for l in range(n):
  for f in range(number_of_filters[l]):
    clip = ImageSequenceClip(list(all_filters[l][f]), fps=10).resize(1.0)
    clip.write_gif(model.layer[layer_indices[l]].name+"_"+"filter"+str(f)+".gif", fps=10)
    print(model.layer[layer_indices[l]].name+"_"+"filter"+str(f)+".gif -- is created")